In [3]:
import time
import tkinter as tk
import pandas as pd
from elev_system.conf.elevator_conf import ELEVLOG_CONFIG

ELEV_LOG = pd.read_csv("./data/elevator_log.csv")

floorList = ['B4','B3','B1','1','2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']
# Elevator ID
elevatorList = ['3001', "3002", "3003", "3004", "3005"]

def cal_floorNum(FloorList):
    top = FloorList[-1]
    btm = FloorList[0]
    
    top = int(top) if not 'B' in top else -int(top[1:]) + 1 
    btm = int(btm) if not 'B' in btm else -int(btm[1:]) + 1
    return abs(top-btm)+1

class colConfig:
    canvas_bg   = "#fff5e0"
    building    = "#404040"
    up_source   = "#f72d77"
    down_source = "#d22df7"
    text = "white"
    
class posConfig: # position config
    window_h = 700
    window_w = 1200
    
    canvas_h = (7./8) * window_h
    canvas_w = (7./8) * window_w
    
    elev_width       = 50
    elev_gap_h       = 20 # horizontal
    
    building_top     = 50
    building_btm     = (4./5) *(canvas_h)
    
    building_wall    = 20
    building_ceiling = 15
    building_base    = 15

    source = 50 # the place where customer wait for elevator
    
    def __init__(self, elevatorNum, floorNum):
        self.building_left = posConfig.canvas_w/2 \
                            - posConfig.building_wall \
                            - posConfig.elev_gap_h * (elevatorNum-1)/2.0 \
                            - posConfig.elev_width * (elevatorNum)/2.0
        
        self.building_right = posConfig.canvas_w/2 + \
                              (posConfig.canvas_w/2 - self.building_left)
        
        self.elev_height = (posConfig.building_btm - posConfig.building_top - \
                            posConfig.building_ceiling - posConfig.building_base) * \
                            ((floorNum + 0.5)/(floorNum*2-1)) / (floorNum)
        
        self.elev_gap_v  = (posConfig.building_btm - posConfig.building_top - \
                            posConfig.building_ceiling - posConfig.building_base) * \
                            ((floorNum-1 - 0.5)/(floorNum*2-1)) / (floorNum-1)# vertical

POS_CONFIG = posConfig(len(elevatorList), cal_floorNum(floorList))

ModuleNotFoundError: No module named 'elev_system'

In [9]:
NOW = 0
SPEED = 10
SMOOTH = 3

class ELEVLOG_CONFIG:
    IDLE = 0
    ARRIVE = 1
    SERVE = 2
    
def delay_by_time(time):
    global NOW, SPEED
    period = time-NOW
    if(period <= 0):
        return 0
    else:
        return int(period * 1000/SPEED) # should be transfer into ms
    
def delay_by_interval(time):
    global SPEED
    return int(time * 1000/SPEED) # should be transfer into ms
    
class Elevator:
    default_floor = 1
    def __init__(self, canvas, name, index, floorList):
        base_floor = int(floorList[0]) if not 'B' in floorList[0] else -int(floorList[0][1:]) + 1
        self.current_floor = 1-base_floor # 1st floort is not definitely on index 0, e.g. ["B1", "1"]
        self.name = name
        
        self.coord1 = [POS_CONFIG.building_left + POS_CONFIG.building_wall + \
                       index * (POS_CONFIG.elev_gap_h + POS_CONFIG.elev_width), 
                       POS_CONFIG.building_btm - POS_CONFIG.building_base - \
                       self.current_floor * (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)]
        
        self.coord2 = [POS_CONFIG.building_left + POS_CONFIG.building_wall + \
                       index * (POS_CONFIG.elev_gap_h + POS_CONFIG.elev_width) + \
                       POS_CONFIG.elev_width, 
                       POS_CONFIG.building_btm - POS_CONFIG.building_base - \
                       self.current_floor * (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height) - \
                       POS_CONFIG.elev_height] 
        
        self.riderNum = 0
        
        self.canvas = canvas
        
        self.elev = self.canvas.create_rectangle(*self.coord1, *self.coord2, fill="#008c9e", width=0)
        self.riderLabel = self.canvas.create_text( \
            self.coord1[0] + (self.coord2[0] - self.coord1[0])/2, 
            self.coord1[1] + (self.coord2[1] - self.coord1[1])/2, 
            text=str(self.riderNum))
        
        global ELEV_LOG
        self.log = ELEV_LOG[ELEV_LOG.name == float(self.name)] #!!!!!!!!!!!!
        print("!!!!!!!!!!!!!!!!!!!!!", self.name)
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!", ELEV_LOG.name.unique())
        self.logPtr = 0
        
        self.draw()
        
    def up(self):
        self.canvas.move(self.riderLabel, 0, -(POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH)
        self.canvas.move(self.elev, 0, -(POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH)
#         self.coord1[1] -= (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH
#         self.coord1[1] -= (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH
    
    def down(self):
        self.canvas.move(self.riderLabel, 0, (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH)
        self.canvas.move(self.elev, 0, (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH) 
#         self.coord1[1] += (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH
#         self.coord1[1] += (POS_CONFIG.elev_gap_v + POS_CONFIG.elev_height)/SMOOTH
    
    def draw(self):
        print(self.log.shape)
        if self.logPtr < self.log.shape[0]: # there exists another action to take # ignore the latest log
            currentAction = self.log.iloc[self.logPtr] 
            nextAction = self.log.iloc[self.logPtr+1]
            if(currentAction["action"] == ELEVLOG_CONFIG.ARRIVE):
                self.current_floor = currentAction["riderNumAfter"]
            elif(currentAction["action"] == ELEVLOG_CONFIG.SERVE):
                self.riderNum = int(currentAction["riderNumAfter"])
                self.canvas.itemconfigure(self.riderLabel, text=str(self.riderNum))
            
            if(nextAction["action"] == ELEVLOG_CONFIG.ARRIVE):
                interval = (float)(nextAction["time"] - currentAction["time"])/3
                for i in range(3):
                    if(nextAction["direction"] == 1):
                        self.canvas.after(delay_by_interval(interval), self.up)
                    elif(nextAction["direction"] == -1):
                        self.canvas.after(delay_by_interval(interval), self.down)
                    else:
                        print(nextAction["direction"])
                        raise Exception("[!] invalid direction")
            self.logPtr += 1
            self.canvas.after(delay_by_time(nextAction["time"]), self.draw)

            


class Source:
    def __init__(self, canvas, elevatorList, floorList):
        self.up   = 0
        self.down = 0

class Building:
    def __init__(self, canvas, elevatorList, floorList):
        self.canvas = canvas
        
        self.elevators = {name:Elevator(self.canvas, name, i, floorList) for i, name in \
                          enumerate(elevatorList)}
        # draw wall
        ## left wall
        self.canvas.create_rectangle(POS_CONFIG.building_left, POS_CONFIG.building_top, 
                                     POS_CONFIG.building_left + POS_CONFIG.building_wall, 
                                     POS_CONFIG.building_btm, 
                                     fill=colConfig.building, width=0)
        
        ## right wall
        self.canvas.create_rectangle(POS_CONFIG.building_right - POS_CONFIG.building_wall, 
                                     POS_CONFIG.building_top, 
                                     POS_CONFIG.building_right, POS_CONFIG.building_btm, 
                                     fill=colConfig.building, width=0)
        ## draw ceiling
        self.canvas.create_rectangle(POS_CONFIG.building_left, POS_CONFIG.building_top, 
                                     POS_CONFIG.building_right, 
                                     POS_CONFIG.building_top + POS_CONFIG.building_ceiling, 
                                     fill=colConfig.building, width=0)
        
        ## draw base
        self.canvas.create_rectangle(POS_CONFIG.building_left,
                                     POS_CONFIG.building_btm - POS_CONFIG.building_base,  
                                     POS_CONFIG.building_right + POS_CONFIG.source, 
                                     POS_CONFIG.building_btm, 
                                     fill=colConfig.building, width=0)
        
        ## draw horizontal gap
        floorNum = cal_floorNum(floorList)
        for i in range(floorNum-1):
            x1 = POS_CONFIG.building_left
            y1 = POS_CONFIG.building_top + POS_CONFIG.building_ceiling + \
                 POS_CONFIG.elev_height * (i+1) + POS_CONFIG.elev_gap_v * (i)
            
            x2 = POS_CONFIG.building_right + POS_CONFIG.source
            y2 = POS_CONFIG.building_top + POS_CONFIG.building_ceiling + \
                 POS_CONFIG.elev_height * (i+1) + POS_CONFIG.elev_gap_v * (i+1)
            
            self.canvas.create_rectangle(x1, y1, x2, y2, fill=colConfig.building, width=0)
        
        ## draw horizontal gap
        for i in range(len(elevatorList)-1):
            x1 = POS_CONFIG.building_left + POS_CONFIG.building_wall + \
                 POS_CONFIG.elev_width * (i+1) + POS_CONFIG.elev_gap_h * (i)
            y1 = POS_CONFIG.building_top
            
            x2 = POS_CONFIG.building_left + POS_CONFIG.building_wall + \
                 POS_CONFIG.elev_width * (i+1) + POS_CONFIG.elev_gap_h * (i+1)
            y2 = POS_CONFIG.building_btm
            
            self.canvas.create_rectangle(x1, y1, x2, y2, fill=colConfig.building, width=0)


class Animation:
    def __init__(self, elevatorList, floorList, title = "Elevator animation"):
        
        self.window = tk.Tk()
        self.window.title(title)
        self.window_w, self.window_h = (posConfig.window_w, posConfig.window_h)
        
#         self.window_w, self.window_h = self.window.maxsize()
#         self.window_h -= 50
        
        self.window.geometry("{}x{}".format(self.window_w, self.window_h)) #注意是string，而不是數字
        self.window.resizable(0,0) #不可以更改大⼩
        
        self.canvas = tk.Canvas(self.window, width=posConfig.canvas_w, height=posConfig.canvas_h, 
                                bg = colConfig.canvas_bg)
        hbar = tk.Scrollbar(self.window, orient = "horizontal")
        hbar.pack(side="bottom",fill='x')
        hbar.config(command=self.canvas.xview)
        
        vbar = tk.Scrollbar(self.window,orient="vertical")
        vbar.pack(side="right",fill='y')
        vbar.config(command=self.canvas.yview)
        
        self.canvas.config(xscrollcommand=hbar.set, yscrollcommand=vbar.set)
        self.canvas.pack(side="left",expand=True,fill="both")


        
        self.closeButton = tk.Button(self.window, text="Shut Down", command=self.destroy)
        self.closeButton.pack()
        
        self.building = Building(self.canvas, elevatorList, floorList)
        
        self.window.after(0, self.time_fleet)
        self.window.mainloop()
        
        
    def time_fleet(self):
        global NOW
        NOW += 1
        self.window.after(delay_by_interval(1), self.time_fleet)
        print(NOW)

    def destroy(self):
        self.window.destroy()

# window.mainloop()
#建立多邊形，頂點坐標（x1,y1,x2,y2,x3,y3），屬於canvas對象，

Animation(elevatorList, floorList)

print("hi")

!!!!!!!!!!!!!!!!!!!!! 3001
!!!!!!!!!!!!!!!!!!!!!!!!!!! [3001 3002 3003 3004]
(117, 7)
!!!!!!!!!!!!!!!!!!!!! 3002
!!!!!!!!!!!!!!!!!!!!!!!!!!! [3001 3002 3003 3004]
(95, 7)
!!!!!!!!!!!!!!!!!!!!! 3003
!!!!!!!!!!!!!!!!!!!!!!!!!!! [3001 3002 3003 3004]
(96, 7)
!!!!!!!!!!!!!!!!!!!!! 3004
!!!!!!!!!!!!!!!!!!!!!!!!!!! [3001 3002 3003 3004]
(100, 7)
!!!!!!!!!!!!!!!!!!!!! 3005
!!!!!!!!!!!!!!!!!!!!!!!!!!! [3001 3002 3003 3004]
(0, 7)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(100, 7)
17
18
19
20
21
22
23
(95, 7)
24
25
26
(100, 7)
27
28
29
30
31
32
(95, 7)
33
34
35
36
(100, 7)
37
38
39
40
41
42
(95, 7)
43
44
(117, 7)
(95, 7)
45
(100, 7)
46
47
48
(100, 7)
49
50
51
52
53
54
(117, 7)
55
(95, 7)
(95, 7)
56
57
58
59
(100, 7)
60
61
62
63
64
(117, 7)
65
66
67
68
69
(100, 7)
70
71
72
73
74
(117, 7)
(117, 7)
(117, 7)
75
76
77
78
(100, 7)
79
80
81
82
83
84
85
86
87
hi


In [195]:
ELEV_LOG

,Unnamed: 0,index,action,riderNumAfter,floor,time,direction
0,0,3001.0,0.0,0.0,1,0.000000,NaN
1,1,3002.0,0.0,0.0,1,0.000000,NaN
2,2,3003.0,0.0,0.0,1,0.000000,NaN
3,3,3004.0,0.0,0.0,1,0.000000,NaN
4,4,3005.0,0.0,0.0,1,0.000000,NaN
...,...,...,...,...,...,...,...
435,435,3001.0,1.0,NaN,2,985.328330,NaN
436,436,3002.0,1.0,NaN,6,986.719349,NaN
437,437,3002.0,2.0,1.0,6,988.719349,1.0
438,438,3001.0,1.0,NaN,1,995.328330,NaN


In [ ]:
from Tkinter import *
import random
import time

tk = Tk()
tk.title = "Game"
tk.resizable(0,0)
tk.wm_attributes("-topmost", 1)

canvas = Canvas(tk, width=500, height=400, bd=0, highlightthickness=0)
canvas.pack()

class Ball:
    def __init__(self, canvas, color):
        self.canvas = canvas
        self.id = canvas.create_oval(10, 10, 25, 25, fill=color)
        self.canvas.move(self.id, 245, 100)

    def draw(self):
        self.canvas.move(self.id, 0, -1)
        self.canvas.after(1, self.draw)  #(time_delay, method_to_execute)




ball = Ball(canvas, "red")
ball.draw()  #Changed per Bryan Oakley's comment
tk.mainloop()